<h1 align="center">Explorar el concepto de Pluralismo utilizando la Ciencia de los Datos: un estudio de caso con el ecosistema mediático de Chile</h3>

<div align="center"><i>Autores: Profesores y Estudiantes del Magíster en Informática (Universidad Austral de Chile)</i></div>
<div align="center"><i>I semestre 2018</i></div>

<h2>1. Objetivo de investigación general</h2>
<ul>
<li><p>El <b>Pluralismo</b> de los medios es un principio que garantiza que l@s ciudadan@s disponen de una información pólitica e ideólogica diversificada, permitiendoles ejercer su <i>espiritu crítico</i> y su <i>libertad de pensar</i>. Por lo tanto, la Organización de las Naciones Unidas para la Educación, la Ciencia y la Cultura (UNESCO) definió el pluralismo de los medios como una condición necesaria para construir la Democracía.</p></li>
<li><b>Medir para entender</b>: <i>“When you can measure what you are speaking about, and express it in numbers, you know something about it; but when you cannot measure it, when you cannot express it in numbers, your knowledge is of a meagre and unsatisfactory kind; it may be the beginning of knowledge, but you have scarcely, in your thoughts, advanced to the stage of science, whatever the matter may be.”</i> - Lord Kelvin (1883)
<li><b>Pregunta general:</b> ¿La Informática, más particularmente la Ciencia de los Datos, puede medir el Pluralismo de los medios? ¿Se puede establecer un protocolo computacional para medir y entender varias facetas del Pluralismo de los Medios basandose sobre técnicas de Clustering de datos?</li>
</ul>

<h2>2. Experimentación de Ciencia de los Datos</h2>

<h3><b>1. Definir una pregunta de investigación </b></h3>

<p><u>Ejemplos de preguntas:</u></p>
<ul>
<li>En el marco de una temática (<i>Mapuche, Cambio climático, Feminismo, Sexismo, etc.</i>), ¿en cuántos tópicos se dividen los discursos de los medios de prensa? ¿Existen diferencias significativas entre los distintos medios? </li>
<li>En el marco de una semana de noticias, ¿en cuántos tópicos se dividen las noticias? ¿Existen diferencias significativas entre los distintos medios?</li>
</ul>


...

<h3>2. Recopilar y preparar los datos</h3>

In [1]:
import numpy as np
import pandas as pd
#Cargar el dataset de tweets
df_feminismo = pd.read_csv('datasets/sophia_mapuche_v2.csv',delimiter="|", header=None)
df_feminismo

#selección de los mensajes
docs = df_feminismo[3].as_matrix()
len(docs)

1493

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import adjusted_rand_score

stemmer = SnowballStemmer("spanish")

# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('spanish')

# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/roberto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/roberto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<h3>3. Explorar los datos</h3>

In [7]:
#Aplicando Modelos Probabilistas de Tópicos y LDA
from sklearn.decomposition import LatentDirichletAllocation
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

tf_vectorizer = TfidfVectorizer(max_df=0.2, min_df=7,
                                stop_words='english',tokenizer=tokenize_only, ngram_range=(1,1))
tf = tf_vectorizer.fit_transform(docs)
diccionario= tf_vectorizer.get_feature_names()


#Estimación de LDA con Bayes Variacional
lda = LatentDirichletAllocation(n_components=10, max_iter=10,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

lda.fit(tf)
#Cálculo de índice de ajuste de los datos
print(lda.perplexity(tf))
print("\nTopics in LDA model:")
print_top_words(lda, diccionario, 30)

10241.54412489984

Topics in LDA model:
Topic #0: newsletter tu correos emol farox guioteca entérate contáctenos quieres hoyxhoy.cl anunciar elige ¿cuándo soychile.cl recibirás inmobiliaria pronto sucede propiedades autolocal.cl suscríbete sa futuro condiciones valor recibir términos periodística sociedad publicado
Topic #1: carabineros comuneros pueblos huelga ley conflicto territorio hambre nos cultura comuna nacional hasta violencia hay pero lugar nuestra años caso tiene días tras dos fueron presos año e indígenas zona
Topic #2: cultura pueblos muy encuentro universidad historia pero nacional alejandro salud huilliche militares hay acuerdos sino tiene guillier cultural dijo poesía machi taller social según obra uno senador violeta dio parciales
Topic #3: lleu koz nuevo tu wiñoy valor tripantu encuentro loading estación poesía junio newsletter parlamento condición raíces hemisferio año frente tags correo invierno los/as organizado día tendrán acuerdos autoridades recuperación daniela